In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
sys.path.append('../resnet_model')

In [2]:
from os.path import join
import os

In [3]:
from os.path import join
import os
import argparse
import importlib
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data import DataLoader
from resnet_model.dataset import SurgicalSingletaskDataset, PredictionDataset, SurgicalMultitaskDataset, SurgicalSingletaskDatasetForParallelFCLayers
from loss import MultiTaskLoss
from custom_transform import CustomTransform
from utils.general.dataset_variables import TripletSegmentationVariables
from resnet_model.models import SingleTaskResNetFPNForParallelFCLayers
import importlib


In [4]:
import sys
import importlib

# Add the parent directory to sys.path so Python can find the module
sys.path.append("../resnet_model/config/triplet_segmentation_test_folder")

# Import the module using its name (without .py)
module_name = "singletask_resnet_fpn_target_parallel_fc"
config = importlib.import_module(module_name)


In [5]:
### 1️⃣ Load Dataset Correctly ###
print("\n🔍 Step 1: Loading dataset...")

# Load instrument-to-task mappings
instrument_to_verb_classes = TripletSegmentationVariables.instrument_to_verb_classes
instrument_to_target_classes = TripletSegmentationVariables.instrument_to_target_classes
instrument_to_verbtarget_classes = TripletSegmentationVariables.instrument_to_verbtarget_classes

transform = CustomTransform(image_size=config.image_size,
                                model_input_size=config.model_input_size)

# Define dataset for the `target` task
target_dataset = SurgicalSingletaskDatasetForParallelFCLayers(
    config=config,  # Your training config
    img_dir=config.train_image_dir,
    ann_for_second_stage_dir=config.train_ann_dir,
    transform=transform,
    train_mode=True
)

# Verify dataset sample
print(f"✅ Loaded dataset: {len(target_dataset)} samples")
sample_img, sample_mask, sample_instr_id, sample_inst_id, sample_local_target_id, sample_name = target_dataset[0]
print(f"🖼️ Sample Data: Img Shape={sample_img.shape}, Mask Shape={sample_mask.shape}, "
      f"Instrument ID={sample_instr_id}, Local Target ID={sample_local_target_id}, Name={sample_name}")

# Create DataLoader
target_dataloader = DataLoader(target_dataset, batch_size=4, shuffle=True)




🔍 Step 1: Loading dataset...
✅ Loaded dataset: 21252 samples
🖼️ Sample Data: Img Shape=torch.Size([3, 224, 224]), Mask Shape=torch.Size([1, 224, 224]), Instrument ID=0, Local Target ID=0, Name=t50_VID01_000000,grasper,1,grasp,gallbladder


In [6]:
print(config.instrument_to_task_classes)

{0: [0, 1, 2, 3, 4, 8, 10, 11, 12, 13, 14], 1: [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14], 2: [0, 1, 2, 3, 4, 5, 8, 10, 11, 14], 3: [0, 1, 2, 3, 5, 8, 9, 10, 11, 14], 4: [1, 2, 3, 4, 5, 14], 5: [0, 1, 2, 4, 6, 7, 8, 10, 14]}


In [7]:
### 2️⃣ Generate Model Correctly ###
print("\n🔍 Step 2: Creating model...")

# Instantiate model
model = SingleTaskResNetFPNForParallelFCLayers( TripletSegmentationVariables.num_instuments, 
                                                     config.instrument_to_task_classes)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("✅ Model successfully loaded on device:", device)


🔍 Step 2: Creating model...


c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tal22\Documents\repositories\triplet_segmentation\.tripletsegmentation\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Model successfully loaded on device: cuda


In [8]:
model

SingleTaskResNetFPNForParallelFCLayers(
  (resnet): ResNet(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downs

In [9]:

### 3️⃣ Pass Data to Model ###
print("\n🔍 Step 3: Passing sample data to model...")

# Get a sample batch from dataloader
sample_batch = next(iter(target_dataloader))
img_batch, mask_batch, instrument_ids_batch, instance_ids_batch, local_target_gt_batch, _ = sample_batch
device = 'cuda'

# Move to device
img_batch = img_batch.to(device)
mask_batch = mask_batch.to(device)
instrument_ids_batch = instrument_ids_batch.to(device)
# Forward pass
task_logits = model(img_batch, mask_batch, instrument_ids_batch)

print(f"✅ Model Output: Logits Shape={task_logits.shape}")



🔍 Step 3: Passing sample data to model...
✅ Model Output: Logits Shape=torch.Size([4, 13])


In [10]:
instrument_to_target_classes

{0: [0, 1, 2, 3, 4, 8, 10, 11, 12, 13, 14],
 1: [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14],
 2: [0, 1, 2, 3, 4, 5, 8, 10, 11, 14],
 3: [0, 1, 2, 3, 5, 8, 9, 10, 11, 14],
 4: [1, 2, 3, 4, 5, 14],
 5: [0, 1, 2, 4, 6, 7, 8, 10, 14]}

In [15]:
instrument_ids_batch

tensor([0, 5, 2, 0], device='cuda:0')

In [14]:

### 4️⃣ Check Model Predictions ###
print("\n🔍 Step 4: Checking model predictions...")

# Convert logits to predicted classes
pred_local_target_ids = torch.argmax(task_logits, dim=1)
print(f"✅ Local Target Predictions: {pred_local_target_ids.tolist()}")

# Convert local -> global task IDs
global_target_preds = [
    instrument_to_target_classes[instr.item()][pred.item()]
    for instr, pred in zip(instrument_ids_batch, pred_local_target_ids)
]
print(f"🌍 Global Target Predictions: {global_target_preds}")


🔍 Step 4: Checking model predictions...
✅ Local Target Predictions: [4, 0, 8, 3]
🌍 Global Target Predictions: [4, 0, 11, 3]
